# Notebook 5: Full Monte Carlo Simulation

**Learning Objectives:**
- Combine market simulation, mortality, and tax strategies
- Run 10,000 simulations comparing strategies
- Store results in DuckDB for analysis
- Answer the key question: Which strategy wins?

In [ ]:
import numpy as np
import duckdb
import matplotlib.pyplot as plt
from pathlib import Path
import sys
sys.path.insert(0, str(Path('.').resolve().parent / 'src'))

from market_simulation import MarketSimulator
from survival_analysis import SurvivalModel
from tax_strategies import HoldToDeathStrategy, AggressiveConversionStrategy
from simulation_engine import SimulationEngine

np.random.seed(42)
print("All modules loaded!")

## Setup: Define Our Retiree

Let's model a 65-year-old male with:
- $1,000,000 in Traditional IRA
- $500,000 in taxable brokerage (cost basis $200,000)
- 24% marginal tax bracket

In [ ]:
# Simulation parameters
config = {
    'start_age': 65,
    'gender': 'M',
    'initial_ira': 1_000_000,
    'initial_taxable': 500_000,
    'cost_basis': 200_000,
    'tax_bracket': 0.24,
    'cap_gains_rate': 0.15,
    'expected_return': 0.07,
    'volatility': 0.16,
    'n_simulations': 10_000
}

print("Simulation Configuration:")
for k, v in config.items():
    print(f"  {k}: {v}")

In [ ]:
# Initialize components
market = MarketSimulator(mu=config['expected_return'], sigma=config['volatility'])
survival = SurvivalModel()

hold_strategy = HoldToDeathStrategy(
    tax_bracket=config['tax_bracket'],
    cap_gains_rate=config['cap_gains_rate']
)

convert_strategy = AggressiveConversionStrategy(
    tax_bracket=config['tax_bracket'],
    cap_gains_rate=config['cap_gains_rate'],
    annual_conversion=100_000,
    conversion_end_age=72
)

print("Components initialized!")

## Run the Simulation

In [ ]:
# Run Monte Carlo
engine = SimulationEngine(market, survival)

print("Running Hold-to-Death simulations...")
hold_results = engine.run_monte_carlo(
    strategy=hold_strategy,
    start_age=config['start_age'],
    gender=config['gender'],
    initial_ira=config['initial_ira'],
    initial_taxable=config['initial_taxable'],
    cost_basis=config['cost_basis'],
    n_simulations=config['n_simulations'],
    seed=42
)

print("Running Aggressive Conversion simulations...")
convert_results = engine.run_monte_carlo(
    strategy=convert_strategy,
    start_age=config['start_age'],
    gender=config['gender'],
    initial_ira=config['initial_ira'],
    initial_taxable=config['initial_taxable'],
    cost_basis=config['cost_basis'],
    n_simulations=config['n_simulations'],
    seed=42
)

print(f"Completed {config['n_simulations']} simulations for each strategy!")

## Store Results in DuckDB

In [ ]:
# Create database and store results
db_path = Path('.').resolve().parent / 'data' / 'simulation_results.duckdb'
db_path.parent.mkdir(exist_ok=True)

con = duckdb.connect(str(db_path))

# Create table
con.execute("""
    CREATE OR REPLACE TABLE simulation_results (
        simulation_id INTEGER,
        strategy VARCHAR,
        death_age INTEGER,
        terminal_wealth DOUBLE,
        total_taxes DOUBLE,
        total_rmds DOUBLE,
        step_up_benefit DOUBLE
    )
""")

# Insert results
for i, r in enumerate(hold_results):
    con.execute("INSERT INTO simulation_results VALUES (?, ?, ?, ?, ?, ?, ?)",
                [i, 'hold_to_death', r['death_age'], r['terminal_wealth'],
                 r['total_taxes'], r['total_rmds'], r['step_up_benefit']])

for i, r in enumerate(convert_results):
    con.execute("INSERT INTO simulation_results VALUES (?, ?, ?, ?, ?, ?, ?)",
                [i, 'aggressive_conversion', r['death_age'], r['terminal_wealth'],
                 r['total_taxes'], r['total_rmds'], r['step_up_benefit']])

print(f"Stored {len(hold_results) + len(convert_results)} results in DuckDB")

## Analyze Results with SQL

In [ ]:
# Summary statistics by strategy
summary = con.execute("""
    SELECT 
        strategy,
        COUNT(*) as n,
        AVG(terminal_wealth) as mean_wealth,
        MEDIAN(terminal_wealth) as median_wealth,
        PERCENTILE_CONT(0.05) WITHIN GROUP (ORDER BY terminal_wealth) as p5,
        PERCENTILE_CONT(0.95) WITHIN GROUP (ORDER BY terminal_wealth) as p95,
        AVG(total_taxes) as mean_taxes
    FROM simulation_results
    GROUP BY strategy
""").fetchdf()

print(summary.to_string(index=False))

In [ ]:
# Which strategy wins more often?
comparison = con.execute("""
    WITH paired AS (
        SELECT 
            h.simulation_id,
            h.terminal_wealth as hold_wealth,
            c.terminal_wealth as convert_wealth
        FROM simulation_results h
        JOIN simulation_results c ON h.simulation_id = c.simulation_id
        WHERE h.strategy = 'hold_to_death' AND c.strategy = 'aggressive_conversion'
    )
    SELECT 
        SUM(CASE WHEN hold_wealth > convert_wealth THEN 1 ELSE 0 END) as hold_wins,
        SUM(CASE WHEN convert_wealth > hold_wealth THEN 1 ELSE 0 END) as convert_wins,
        AVG(convert_wealth - hold_wealth) as avg_difference
    FROM paired
""").fetchone()

print(f"Hold-to-Death wins: {comparison[0]:,} times ({comparison[0]/config['n_simulations']*100:.1f}%)")
print(f"Aggressive Conversion wins: {comparison[1]:,} times ({comparison[1]/config['n_simulations']*100:.1f}%)")
print(f"Average difference: ${comparison[2]:,.0f}")

In [ ]:
con.close()
print(f"Results saved to: {db_path}")

## Summary

We've built a complete Monte Carlo simulation that:
1. Simulates market returns using GBM
2. Samples death ages from SSA life tables
3. Applies different tax strategies
4. Stores results in DuckDB for analysis

**Key Insight:** The optimal strategy depends on how long you live and market performance!